<a href="https://colab.research.google.com/github/MissoumYoucef/Kaggle_Competition/blob/main/learning_agency_lab_automated_essay_scoring.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'learning-agency-lab-automated-essay-scoring-2:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-competitions-data%2Fkaggle-v2%2F71485%2F8059942%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240608%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240608T173248Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D73b75b7fb3f6996d34b0c40293a2c4f168792d5edc5300f039110076cca426fb68246483b55dc456338aa4d7c7e0b7c0739019edc2767061077517d47e5d7913ed1eae3b8e04ac9c6e2d979c23d3c62118a5053684732fd2cf7ffa6d35382e72dbbb3cebb1098f25f9e8e316a119abc1aeb1897f3cb9a9d3981964502509d1e14e6cc5686a874f49cf8d85fe6ffa1e22e0bf7726971d1203452d8fa78c90e936cc376124fd1817c67ce0ede22debcd3be9da65afd50ddf2872bae9c7dad16bdd4a084c445b560e59622d4eaf4e2ef8520adddd0889aa5d97e92e3c10bbff4766c0a31e5504b3fce45f8960176c0c53895ba29ebaac17d5abb2daaa3e4caed575'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/learning-agency-lab-automated-essay-scoring-2/sample_submission.csv
/kaggle/input/learning-agency-lab-automated-essay-scoring-2/train.csv
/kaggle/input/learning-agency-lab-automated-essay-scoring-2/test.csv


In [ ]:
train_df = pd.read_csv('/kaggle/input/learning-agency-lab-automated-essay-scoring-2/train.csv')
test_df = pd.read_csv('/kaggle/input/learning-agency-lab-automated-essay-scoring-2/test.csv')

In [ ]:
train_df.shape

(17307, 3)

In [ ]:
train_df.columns

Index(['essay_id', 'full_text', 'score'], dtype='object')

In [ ]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17307 entries, 0 to 17306
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   essay_id   17307 non-null  object
 1   full_text  17307 non-null  object
 2   score      17307 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 405.8+ KB


In [ ]:
train_df.head()

,essay_id,full_text,score
0,000d118,Many people have car where they live. The thin...,3
1,000fe60,I am a scientist at NASA that is discussing th...,3
2,001ab80,People always wish they had the same technolog...,4
3,001bdc0,"We all heard about Venus, the planet without a...",4
4,002ba53,"Dear, State Senator\n\nThis is a letter to arg...",3


In [ ]:
# train_df['score'].unique()

array([3, 4, 2, 1, 5, 6])

In [ ]:
# # Create a new column 'word_count'
# word_count = train_df['full_text'].apply(lambda text: len(str(text).split()))

In [ ]:
# word_count.unique()

In [ ]:
# len(word_count.unique())

793

In [ ]:
# train_df['word_count']=word_count
# train_df.columns

Index(['essay_id', 'full_text', 'score', 'word_count'], dtype='object')

In [ ]:
# train_df = pd.get_dummies(train_df, columns=['score'], prefix='score')

In [ ]:
from sklearn.model_selection import train_test_split
# Split the data into training and validation sets
train_texts, val_texts, train_scores, val_scores = train_test_split(
    train_df['full_text'],
    train_df['score'],
    test_size=0.2,
    random_state=42
)

# **TF_IDF**

In [ ]:
# # Import necessary libraries
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.linear_model import LogisticRegression
# from sklearn.metrics import cohen_kappa_score, make_scorer

In [ ]:
# # Create a TF-IDF vectorizer to convert text data into numerical features
# vectorizer = TfidfVectorizer(max_features=5000)
# train_features = vectorizer.fit_transform(train_texts)
# val_features = vectorizer.transform(val_texts)

# **BOW**

In [ ]:
# from sklearn.feature_extraction.text import CountVectorizer

# vectorizer = CountVectorizer(max_features=5000)  # You can adjust max_features
# train_features = vectorizer.fit_transform(train_texts)
# val_features = vectorizer.transform(val_texts)

# **Word Embeddings:**

In [ ]:
# import gensim  # You'll need to install gensim: pip install gensim

# # Load a pre-trained Word2Vec model (or train your own)
# model = gensim.models.Word2Vec.load('path/to/word2vec/model')

# def get_doc_vector(text):
#     words = text.split()
#     vectors = [model.wv[word] for word in words if word in model.wv]
#     if vectors:
#         return np.mean(vectors, axis=0)
#     else:
#         return np.zeros(model.vector_size) # Return a zero vector if no words are found

# train_features = np.array([get_doc_vector(text) for text in train_texts])
# val_features = np.array([get_doc_vector(text) for text in val_texts])

# **Bert**

In [ ]:
# from transformers import BertTokenizer, BertModel
# import torch

# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# model = BertModel.from_pretrained('bert-base-uncased')

In [ ]:
# # Function to get BERT embeddings
# def get_bert_embedding(text):
#     input_ids = tokenizer.encode(text, add_special_tokens=True)
#     input_ids = torch.tensor([input_ids])
#     with torch.no_grad():
#         outputs = model(input_ids)
#         last_hidden_states = outputs.last_hidden_state
#     return last_hidden_states[0][0].numpy() # Get the [CLS] token embedding

# train_features = np.array([get_bert_embedding(text) for text in train_texts])
# val_features = np.array([get_bert_embedding(text) for text in val_texts])

# **DATA Shape**

In [ ]:
# train_features.shape,train_features[0].shape,val_features.shape,val_features[0].shape

((13845, 5000), (1, 5000), (3462, 5000), (1, 5000))

# **LogisticRegression**

In [ ]:
# # Train a logistic regression model
# model = LogisticRegression(max_iter=1000, random_state=42)
# model.fit(train_features, train_scores)

# # Make predictions on the validation set
# val_predictions = model.predict(val_features)

# # Evaluate the model using quadratic weighted kappa
# kappa = cohen_kappa_score(val_scores, val_predictions, weights='quadratic')
# print(f'Quadratic Weighted Kappa: {kappa:.4f}')

# **SVC**

In [ ]:
# from sklearn.svm import LinearSVC  # For linear SVM
# from sklearn.svm import SVC  # For non-linear SVM

# model = LinearSVC(max_iter=1000, random_state=42)
# # model = SVC(kernel='rbf', random_state=42) # For non-linear
# model.fit(train_features, train_scores)
# val_predictions = model.predict(val_features)

# # Evaluate the model using quadratic weighted kappa
# kappa = cohen_kappa_score(val_scores, val_predictions, weights='quadratic')
# print(f'Quadratic Weighted Kappa: {kappa:.4f}')

Quadratic Weighted Kappa: 0.5302


/opt/conda/lib/python3.10/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


# **RandomForestClassifier**

In [ ]:
# from sklearn.ensemble import RandomForestClassifier

# model = RandomForestClassifier(n_estimators=100, random_state=42)
# model.fit(train_features, train_scores)
# val_predictions = model.predict(val_features)
# # Evaluate the model using quadratic weighted kappa
# kappa = cohen_kappa_score(val_scores, val_predictions, weights='quadratic')
# print(f'Quadratic Weighted Kappa: {kappa:.4f}')

Quadratic Weighted Kappa: 0.6334


# **LSTM**

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

# Load your data
Train_df = train_df

# Preprocess text data
# 1. Tokenization
tokenizer = Tokenizer(num_words=5000)  # Adjust num_words as needed
tokenizer.fit_on_texts(Train_df['full_text'])
sequences = tokenizer.texts_to_sequences(Train_df['full_text'])

# 2. Padding
max_length = 800  # Adjust max_length as needed
padded_sequences = pad_sequences(sequences, maxlen=max_length, padding='post', truncating='post')

# 3. Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, Train_df['score'], test_size=0.2)

# Normalize scores (if they are 1, 2, ..., 6)
y_train = y_train - 1
y_test = y_test - 1

# Create the model
vocab_size = len(tokenizer.word_index) + 1
embedding_dim = 100

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_length))
model.add(LSTM(128, return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(64))
model.add(Dropout(0.2))
model.add(Dense(6, activation='softmax'))

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',  # For categorical scores
              metrics=['accuracy'])

# Early Stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test),
          callbacks=[early_stopping])

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}')
print(f'Test Accuracy: {accuracy}')

2024-06-08 17:26:19.766581: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-08 17:26:19.766681: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-08 17:26:19.903377: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/10
433/433 ━━━━━━━━━━━━━━━━━━━━ 42s 83ms/step - accuracy: 0.3536 - loss: 1.4846 - val_accuracy: 0.3749 - val_loss: 1.4337
Epoch 2/10
433/433 ━━━━━━━━━━━━━━━━━━━━ 36s 82ms/step - accuracy: 0.3629 - loss: 1.4383 - val_accuracy: 0.3752 - val_loss: 1.4300
Epoch 3/10
433/433 ━━━━━━━━━━━━━━━━━━━━ 36s 82ms/step - accuracy: 0.3869 - loss: 1.3471 - val_accuracy: 0.5404 - val_loss: 1.1295
Epoch 4/10
433/433 ━━━━━━━━━━━━━━━━━━━━ 36s 83ms/step - accuracy: 0.5333 - loss: 1.1001 - val_accuracy: 0.5621 - val_loss: 1.0171
Epoch 5/10
433/433 ━━━━━━━━━━━━━━━━━━━━ 36s 83ms/step - accuracy: 0.5704 - loss: 1.0311 - val_accuracy: 0.5350 - val_loss: 1.0525
Epoch 6/10
219/433 ━━━━━━━━━━━━━━━━━━━━ 16s 75ms/step - accuracy: 0.5928 - loss: 1.0002

In [ ]:
# val_predictions = model.predict_classes(val_features)
# Prepare the test data for prediction
test_features = vectorizer.transform(test_df['full_text'])

# Make predictions on the test data
test_predictions = model.predict(test_features)

# Create a submission file
submission_df = pd.DataFrame({'essay_id': test_df['essay_id'], 'score': test_predictions})
submission_df.to_csv('submission.csv', index=False)

print("Submission file created successfully!")

Submission file created successfully!
